In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
import random
import cv2
import numpy as np
!pip install pyefd
import pyefd
from google.colab.patches import cv2_imshow
!pip install cairocffi
import cairocffi as cairo
import struct
from struct import unpack

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 7.4 MB/s 
  Created wheel for cairocffi: filename=cairocffi-1.3.0-py3-none-any.whl size=89668 sha256=08f47622cfcda607a3048a93a62799e66f209280b212e75a9bc419e5592c8370
  Stored in directory: /root/.cache/pip/wheels/4e/ca/e1/5c8a9692a27f639a07c949044bec943f26c81cd53d3805319f
Successfully built cairocffi


In [3]:
# Env vars
torch.use_deterministic_algorithms(False)

# Const vars
LOAD_PATH = '/content/drive/My Drive/Fourier/Saved Models/MLP raster single.pt'
RAND_SEED = 0
DEVICE = "cuda"

IMG_SIDE = 28
NUM_CLASSES = 3
EPOCHS = 100
BATCH_SIZE = 512
LOSS_FN = nn.MSELoss()

In [43]:
# convert raw vector image to single raster image
def vector_to_raster(vector_image, side=IMG_SIDE, line_diameter=16, padding=96, bg_color=(0,0,0), fg_color=(1,1,1)):
  """
  padding and line_diameter are relative to the original 256x256 image.
  """
  
  original_side = 256.
  
  surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, side, side)
  ctx = cairo.Context(surface)
  ctx.set_antialias(cairo.ANTIALIAS_BEST)
  ctx.set_line_cap(cairo.LINE_CAP_ROUND)
  ctx.set_line_join(cairo.LINE_JOIN_ROUND)
  ctx.set_line_width(line_diameter)

  # scale to match the new size
  # add padding at the edges for the line_diameter
  # and add additional padding to account for antialiasing
  total_padding = padding * 2. + line_diameter
  new_scale = float(side) / float(original_side + total_padding)
  ctx.scale(new_scale, new_scale)
  ctx.translate(total_padding / 2., total_padding / 2.)
      
  bbox = np.hstack(vector_image).max(axis=1)
  offset = ((original_side, original_side) - bbox) / 2.
  offset = offset.reshape(-1,1)
  centered = [stroke + offset for stroke in vector_image]

  # clear background
  ctx.set_source_rgb(*bg_color)
  ctx.paint()

  # draw strokes, this is the most cpu-intensive part
  ctx.set_source_rgb(*fg_color)     
  for xv, yv in centered:   
    ctx.move_to(xv[0], yv[0])
    for x, y in zip(xv, yv):
        ctx.line_to(x, y)
    ctx.stroke()

  data = surface.get_data()
  raster = np.copy(np.asarray(data)[::4]).reshape(28, 28)
  return raster

# Define transformation(s) to be applied to dataset-
transforms_norm = T.Compose(
      [
          T.ToTensor(), # scales integer inputs in the range [0, 255] into the range [0.0, 1.0]
      ]
  )

means = [1.35058459e+01, 1.41851824e+01, -9.79489986e-01, -6.33104260e+00,
 -6.49051018e+00,  9.49529280e-01, -3.48707740e-02,  4.53843623e-01,
 -6.14641528e-01,  5.13467360e-02, -3.29423276e-02,  2.43653038e-02,
 -1.14767802e-01, -6.08045509e-02, -7.58547497e-03, -7.11006358e-02,
 -1.74108440e-01,  2.41459887e-02, -3.56433303e-02,  7.45719611e-05,
 -1.13045070e-01,  2.30913050e-02, -6.37490433e-03,  4.31639346e-03,
 -4.86658106e-02,  1.93030196e-02, -6.75940589e-03, -1.16378550e-02,
 -6.01807952e-02,  3.93879263e-03, -9.34910813e-03,  9.02253774e-03,
 -3.41462722e-02,  1.38882049e-02, -6.11831384e-03, -7.83896568e-03,
 -3.71848393e-02,  1.11462354e-02, -4.61157723e-03, -4.44754610e-04,
 -2.65179814e-02,  1.13610795e-02, -3.89029941e-03,  1.64774224e-03,
 -2.47375372e-02,  7.55868545e-03, -3.79064692e-03,  1.43704716e-03,
 -2.05986010e-02,  8.73466456e-03, -3.29903275e-03, -1.90183118e-03,
 -1.93036681e-02,  7.72213796e-03, -3.52263611e-03,  1.66164771e-03,
 -1.73029018e-02,  6.36046576e-03, -3.36707949e-03,  5.22499707e-04,
 -1.49967296e-02,  3.98918288e-03, -3.91103451e-03,  5.47056152e-04,
 -1.46523840e-02,  2.97338771e-03, -3.22989804e-03,  3.47155201e-04,
 -1.31365258e-02,  2.82104974e-03, -2.05074767e-03, -1.99756328e-04,
 -1.16087027e-02,  1.22665637e-03, -2.43683685e-03,  4.22197931e-05,
 -1.07137717e-02,  1.22258394e-03, -2.09530219e-03,  2.70200940e-04,
 -9.13769261e-03,  8.48098798e-05, -2.09274405e-03, -6.34421711e-04,
 -8.97263423e-03, -2.25563941e-04, -1.74597188e-03,  8.29755361e-04,
 -7.84957888e-03,  1.86340838e-04, -2.05422470e-03,  4.35070320e-04,
 -6.44060467e-03, -8.48069330e-04, -1.38073207e-03, -7.01999658e-04,
 -6.56643950e-03, -6.41538560e-04, -1.20805737e-03, -1.42317359e-04,
 -5.52600824e-03, -7.71141986e-04, -1.17922416e-03,  2.76561875e-04,
 -4.83444806e-03, -9.18393419e-04, -1.12409165e-03,  1.82602934e-04,
 -4.30918623e-03, -1.13872967e-03, -1.26684393e-03,  4.48641045e-05,
 -4.18062019e-03, -9.56536631e-04, -7.67604751e-04, -1.04270797e-04,
 -3.26866673e-03, -8.81154629e-04, -9.31615292e-04, -2.64058046e-05,
 -3.07037421e-03, -9.00031217e-04, -6.32932184e-04, -4.46136973e-05,
 -2.60526518e-03, -5.58089340e-04, -6.64699377e-04,  2.65900891e-05,
 -2.32977561e-03, -7.17302351e-04, -5.31805919e-04,  6.18960220e-05,
 -2.34882871e-03, -5.65139477e-04, -4.13379584e-04,  1.19149304e-04,
 -2.11239847e-03, -4.86481964e-04, -5.22856561e-04,  8.06273854e-05,
 -1.75490294e-03, -3.31008275e-04, -6.44977667e-04, -1.34050918e-05,
 -1.79682755e-03, -1.84199377e-04, -3.51541979e-04, -9.74131531e-06,
 -1.71028164e-03, -1.46753575e-04, -5.44878363e-04,  8.50742347e-05,
 -1.56292352e-03, -1.29815354e-04, -4.66005452e-04, -7.62196757e-05,
 -1.49752705e-03,  6.83185311e-05, -3.43396558e-04,  1.44167176e-06,
 -1.53783908e-03,  4.98785977e-05]

stdevs = [6.27881640e-01, 8.34754488e-01, 3.03374386e+00, 1.48834951e+00,
 1.53313765e+00, 2.40549841e+00, 6.73943066e-01, 8.12936068e-01,
 9.28296310e-01, 4.29025602e-01, 5.22980465e-01, 4.04179494e-01,
 4.18664460e-01, 4.56595860e-01, 1.91167205e-01, 2.15219149e-01,
 2.25372670e-01, 1.58235754e-01, 1.73654995e-01, 1.90393742e-01,
 1.59446245e-01, 1.36070632e-01, 9.69622939e-02, 1.00633995e-01,
 9.35408652e-02, 8.92466501e-02, 9.46049569e-02, 8.86695724e-02,
 8.42337669e-02, 8.78300361e-02, 6.65550097e-02, 6.65057180e-02,
 5.95213907e-02, 5.82373569e-02, 6.52688968e-02, 6.40227296e-02,
 5.60041096e-02, 5.39057684e-02, 5.03966173e-02, 4.94184529e-02,
 4.54881111e-02, 4.35549194e-02, 4.40808662e-02, 4.59915010e-02,
 4.21086401e-02, 4.24830716e-02, 4.00076674e-02, 3.84023555e-02,
 3.64912515e-02, 3.46676838e-02, 3.70479131e-02, 3.66812046e-02,
 3.22136930e-02, 3.34775544e-02, 3.22718667e-02, 3.21941657e-02,
 2.86306123e-02, 2.89266575e-02, 3.01704759e-02, 3.05456015e-02,
 2.68266946e-02, 2.72623004e-02, 2.75719104e-02, 2.73874682e-02,
 2.44078993e-02, 2.40762505e-02, 2.60451590e-02, 2.61034715e-02,
 2.25770668e-02, 2.18581369e-02, 2.34849276e-02, 2.38630195e-02,
 2.06289107e-02, 2.03209188e-02, 2.24720851e-02, 2.18124532e-02,
 1.90425535e-02, 1.85577903e-02, 2.04656203e-02, 1.98523102e-02,
 1.73252706e-02, 1.63287412e-02, 1.88122306e-02, 1.74965416e-02,
 1.60525395e-02, 1.53401663e-02, 1.71975891e-02, 1.57101456e-02,
 1.43031540e-02, 1.33443356e-02, 1.53122541e-02, 1.41537694e-02,
 1.28646460e-02, 1.24942759e-02, 1.33223768e-02, 1.31047936e-02,
 1.13899621e-02, 1.12663446e-02, 1.15449441e-02, 1.17539317e-02,
 1.03971641e-02, 1.05375924e-02, 1.08966208e-02, 1.07490700e-02,
 9.35980742e-03, 8.98703778e-03, 9.36980941e-03, 9.43069518e-03,
 8.67232745e-03, 8.07232368e-03, 8.59676189e-03, 8.21645706e-03,
 7.39947150e-03, 7.50961076e-03, 7.35765514e-03, 7.22652355e-03,
 6.81490965e-03, 6.63858175e-03, 6.45865217e-03, 6.53639933e-03,
 6.07089017e-03, 5.97741345e-03, 5.95414540e-03, 5.84498405e-03,
 5.58836878e-03, 5.43748702e-03, 5.39396627e-03, 5.27361495e-03,
 5.29277211e-03, 4.85720825e-03, 4.92926728e-03, 4.77341513e-03,
 4.82018374e-03, 4.43140461e-03, 4.39952588e-03, 4.33664615e-03,
 4.42986857e-03, 4.12455852e-03, 4.06600591e-03, 3.91950786e-03,
 4.02053803e-03, 3.82310809e-03, 3.76183601e-03, 3.77341405e-03,
 3.96346934e-03, 3.57321419e-03, 3.54841842e-03, 3.68966722e-03,
 3.83598035e-03, 3.40797874e-03, 3.59240702e-03, 3.68940218e-03,
 3.53164920e-03, 3.33787278e-03, 3.47881487e-03, 3.52159614e-03,
 3.27608497e-03, 2.99042730e-03, 3.41900987e-03, 3.35859693e-03,
 3.30106115e-03, 3.03578054e-03]

# transform functions - take sketch image, return torch tensor of descriptors
def transform(vector_img, is_test):
  raster = vector_to_raster(vector_img)

  # add rotations and translations at test time
  if is_test: 
    raster = transforms_norm(raster.astype(np.float32))

    angle = random.random()*60 - 30
    deltaX = random.randint(-3, 3)
    deltaY = random.randint(-3, 3)

    raster = T.functional.affine(raster, angle, [deltaX, deltaY], 1, 0,
                                 interpolation=T.InterpolationMode.BILINEAR)
    raster = np.squeeze(raster.numpy()).astype(np.uint8)
  
  raster_binary = cv2.threshold(raster, 100, 255, cv2.THRESH_BINARY)[1]
  contours, hierarchy = cv2.findContours(raster.astype(np.uint8), 
                                         cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  
  largest_size = 0
  largest_index = 0
  for k, contour in enumerate(contours):
      if len(contour) > largest_size:
        largest_size = len(contour)
        largest_index = k

  contour = np.asarray(contours[largest_index]).squeeze()

  coeffs = pyefd.elliptic_fourier_descriptors(contour, order=FOURIER_ORDER, normalize=True)
  A0, C0 = pyefd.calculate_dc_coefficients(contour)
  coeffs = coeffs.flatten()
  coeffs = np.concatenate([np.asarray([A0, C0]), coeffs])
  # coeffs = (coeffs - means[0:FOURIER_ORDER * 4 + 2]) / stdevs[0:FOURIER_ORDER * 4 + 2]

  background = np.zeros((IMG_SIDE, IMG_SIDE), dtype=np.float32)
  reconstruction = cv2.drawContours(background, [contour], 0, 1)
  reconstruction = reconstruction * 2 - 1

  return torch.from_numpy(coeffs).float(), torch.from_numpy(reconstruction).unsqueeze(0)

# deterministic worker re-seeding
def seed_worker(worker_id):
  worker_seed = torch.initial_seed() % 2**32
  np.random.seed(worker_seed)
  random.seed(worker_seed)

# custom dataset for quickdraw
class QuickdrawDataset(Dataset):
  def __init__(self, imgs, nums, is_test):
    self.imgs = imgs
    self.nums = nums
    self.len = sum(nums)
    self.is_test = is_test

  def __len__(self):
    return self.len

  def __getitem__(self, idx):
    img = self.imgs[idx]
    x, y = transform(img, self.is_test)
    return x, y


class MLP(torch.nn.Module):
  def __init__(self):
    super(MLP, self).__init__()
    self.fc1 = nn.Linear(FOURIER_ORDER * 4 + 2, 464)
    self.fc2 = nn.Linear(464, 464)
    self.fc3 = nn.Linear(464, 464)
    self.fc4 = nn.Linear(464, 4 * 14 * 14)
    self.upscale = nn.Upsample(scale_factor=2, mode='bilinear')
    self.conv1 = nn.Conv2d(4, 2, 3, padding='same')
    self.conv2 = nn.Conv2d(2, 1, 3, padding='same')
    self.relu = nn.ReLU()
    self.tanh = nn.Tanh()


  def forward(self, x):
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    x = self.relu(x)
    x = self.fc3(x)
    x = self.relu(x)
    x = self.fc4(x)
    x = x.view(-1, 4, 14, 14)
    x = self.relu(x)
    x = self.upscale(x)
    x = self.conv1(x)
    x = self.relu(x)
    x = self.conv2(x)
    return self.tanh(x)


def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train() # put the model in train mode
    total_loss = 0
    # for each batch in the training set compute loss and update model parameters
    for batch, (x, y) in enumerate(dataloader):
      x, y = x.to(DEVICE), y.to(DEVICE)
      # Compute prediction and loss
      out = model(x)
      loss = loss_fn(out, y)

      # Backpropagation to update model parameters
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # print current training metrics for user
      y, out, loss = y.detach().to("cpu"), out.detach().to("cpu"), loss.detach().to("cpu")
      loss_val = loss.item()
      if batch % 10 == 0:
          current = (batch + 1) * BATCH_SIZE
          print(f"loss: {loss_val:>7f}  [{current:>5d}/{size:>5d}]")
          y = (y[0, 0, :, :].numpy() + 1) * 255 / 2
          pred = (out[0, 0, :, :].numpy() + 1) * 255 / 2
          cv2_imshow(pred)
          cv2_imshow(y)
      total_loss += loss_val
      # print(f"train loss: {loss_val:>7f}   train accuracy: {correct / BATCH_SIZE:>7f}   [batch: {batch + 1:>3d}/{(size // BATCH_SIZE) + 1:>3d}]")      
    print(f"\nepoch avg train loss: {total_loss / ((size // BATCH_SIZE) + 1):>7f}")


In [33]:
# define methods for unpacking Quickdraw .bin files
def unpack_drawing(file_handle):
  file_handle.read(15)
  n_strokes, = unpack('H', file_handle.read(2))
  image = []
  for i in range(n_strokes):
      n_points, = unpack('H', file_handle.read(2))
      fmt = str(n_points) + 'B'
      x = unpack(fmt, file_handle.read(n_points))
      y = unpack(fmt, file_handle.read(n_points))
      image.append((x, y))

  return image


def unpack_drawings(filename):
  imageset = []
  with open(filename, 'rb') as f:
      while True:
          try:
              imageset.append(unpack_drawing(f))
          except struct.error:
              break
  return imageset

train_dir = '/content/drive/My Drive/Fourier/Quickdraw Dataset/Train/'
test_dir = '/content/drive/My Drive/Fourier/Quickdraw Dataset/Test/'
train_imgs = []
test_imgs = []
train_nums = []
test_nums = []
list_of_classes = ["The Eiffel Tower", "The Great Wall of China", "The Mona Lisa",
                   "aircraft carrier", "airplane", "alarm clock", "ambulance", 
                   "angel", "ant", "anvil", "apple", "arm", "asparagus", "axe", 
                   "backpack", "banana", "bandage", "barn", "baseball bat", 
                   "baseball", "basket", "basketball", "bathtub", "beach", "bear", 
                   "beard", "bed", "bee", "belt", "bench", "bicycle", "binoculars", 
                   "bird", "birthday cake", "blackberry", "blueberry", "book", 
                   "boomerang", "bottlecap", "bowtie", "bracelet", "brain", 
                   "bread", "bridge", "broccoli", "broom", "bucket", "bulldozer", 
                   "bus", "bush", "butterfly", "cactus", "cake", "calculator", 
                   "calendar", "camel", "camera", "camouflage", "campfire", 
                   "candle", "cannon", "canoe", 'car', 'carrot', "castle", "cat", "ceiling fan", 
                   "cell phone", "cello", "chair", "chandelier", "church", 
                   "circle", "clarinet", "clock", "cloud", "coffee cup", 
                   "compass", "computer", "cookie", "cooler", "couch", "cow",
                   "crab", "crayon", "crocodile", "crown", "cruise ship", 
                   "cup", "diamond", "dishwasher", "diving board", "dog", 
                   "dolphin", "donut", "door", "dragon", "dresser", "drill", 
                   "drums", "duck", "dumbbell", "ear", "elbow", "elephant", 
                   "envelope", "eraser", "eye", "eyeglasses", "face", "fan",
                   "feather", "fence", "finger", "fire hydrant", "fireplace",
                   "firetruck", "fish", "flamingo", "flashlight", "flip flops", 
                   "floor lamp", "flower", "flying saucer", "foot", "fork", 
                   "frog", "frying pan", "garden hose", "garden", "giraffe", 
                   "goatee", "golf club", "grapes", "grass", "guitar", 
                   "hamburger", "hammer", "hand", "harp", "hat", "headphones", 
                   "hedgehog", "helicopter", "helmet", "hexagon", "hockey puck", 
                   "hockey stick", "horse", "hospital", "hot air balloon", 
                   "hot dog", "hot tub", "hourglass", "house plant", "house", 
                   "hurricane", "ice cream", "jacket", "jail", "kangaroo", 
                   "key", "keyboard", "knee", "knife", "ladder", "lantern", 
                   "laptop", "leaf", "leg", "light bulb", "lighter", "lighthouse",
                   "lightning", "line", "lion", "lipstick", "lobster", "lollipop",
                   "mailbox", "map", "marker", "matches", "megaphone", "mermaid", 
                   "microphone", "microwave", "monkey", "moon", "mosquito", 
                   "motorbike", "mountain", "mouse", "moustache", "mouth", "mug",
                   "mushroom", "nail", "necklace", "nose", "ocean", "octagon", 
                   "octopus", "onion", "oven", "owl", "paint can", "paintbrush", 
                   "palm tree", "panda", "pants", "paper clip", "parachute", 
                   "parrot", "passport", "peanut", "pear", "peas", "pencil", 
                   "penguin", "piano", "pickup truck", "picture frame", "pig", 
                   "pillow", "pineapple", "pizza", "pliers", "police car", 
                   "pond", "pool", "popsicle", "postcard", "potato", 
                   "power outlet", "purse", "rabbit", "raccoon", "radio", 
                   "rain", 'rainbow', 'rake', 'remote control', 'rhinoceros', 
                   'rifle', 'river', 'roller coaster', 'rollerskates', 
                   'sailboat', 'sandwich', 'saw', 'saxophone', 'school bus', 
                   'scissors', 'scorpion', 'screwdriver', 'sea turtle', 
                   'see saw', 'shark', 'sheep', 'shoe', 'shorts', 'shovel', 
                   'sink', 'skateboard', 'skull', 'skyscraper', 'sleeping bag', 
                   'smiley face', 'snail', 'snake', 'snorkel', 'snowflake', 
                   'snowman', 'soccer ball', 'sock', 'speedboat', 'spider', 
                   'spoon', 'spreadsheet', 'square', 'squiggle', 'squirrel', 
                   'stairs', 'star', 'steak', 'stereo', 'stethoscope', 'stitches', 
                   'stop sign', 'stove', 'strawberry', 'streetlight', 
                   'string bean', 'submarine', 'suitcase', 'sun', 'swan', 
                   'sweater', 'swing set', 'sword', 'syringe', 't-shirt', 
                   'table', 'teapot', 'teddy-bear', 'telephone', 'television', 
                   'tennis racquet', 'tent', 'tiger', 'toaster', 'toe', 'toilet', 
                   'tooth', 'toothbrush', 'toothpaste', 'tornado', 'tractor', 
                   'traffic light', 'train', 'tree', 'triangle', 'trombone', 
                   'truck', 'trumpet', 'umbrella', 'underwear', 'van', 'vase', 'violin', 
                   'washing machine', 'watermelon', 'waterslide', 'whale', 
                   'wheel', 'windmill', 'wine bottle', 'wine glass', 'wristwatch', 
                   'yoga', 'zebra', 'zigzag']

In [38]:
# load dataset
for item in list_of_classes:
  train_folder = train_dir + item + '.bin'
  # test_folder = test_dir + item + '.bin'
  train_drawings = unpack_drawings(train_folder)
  train_imgs += train_drawings
  train_nums.append(len(train_drawings))
  # test_drawings = unpack_drawings(test_folder)
  # test_imgs += test_drawings
  # test_nums.append(len(test_drawings))

In [44]:
FOURIER_ORDER = 10

# seed RNGs
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)

# create datasets
train_data = QuickdrawDataset(train_imgs, train_nums, is_test=False)
# eval_data = QuickdrawDataset(test_imgs, test_nums, is_test=False)
# test_data = QuickdrawDataset(test_imgs, test_nums, is_test=True)

# create dataloaders
g = torch.Generator()
g.manual_seed(RAND_SEED)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
# eval_loader = DataLoader(eval_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)
# test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)

# init model and optimizer
model = MLP()
# checkpoint = torch.load(LOAD_PATH, map_location=torch.device(DEVICE))
# model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
lr = 0.001
optim = torch.optim.Adam(model.parameters(), lr=lr)
# optim.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
epoch = 0

# train for EPOCHS number of epochs then evaluate on test data with affine transformations
# eval_loop(dataloader=test_loader,model=model)
SAVE_PATH = '/content/drive/My Drive/Fourier/Saved Models/MLP fourier raster reconstruction10.pt'
for i in range(epoch, EPOCHS):
    print("Epoch " + str(i + 1) + "\n")
    train_loop(dataloader=train_loader,model=model,loss_fn=LOSS_FN,optimizer=optim)
    # eval_loop(dataloader=eval_loader,model=model)
    torch.save({
                'epoch': i + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optim.state_dict()
                }, SAVE_PATH)
    print("\n-------------------------------\n")

Epoch 1

loss: 0.883522  [  512/3459879]


loss: 0.867150  [ 5632/3459879]


loss: 0.849409  [10752/3459879]


loss: 0.788144  [15872/3459879]


loss: 0.586654  [20992/3459879]


...

loss: 0.186852  [763392/3459879]


loss: 0.182502  [768512/3459879]


loss: 0.180721  [773632/3459879]


loss: 0.183047  [778752/3459879]


KeyboardInterrupt: ignored